# SHRUMS Data Wrangling 5 channels

## Jupyter notebook implementation

This is an implementation of the SHRUMS data wrangling in python. As CellProfiler processes the images, it will detect the nuclei in each Z slice, and then combine them using tracking.  The function of this script is to integrate the the data from each 3D nucleus across images in a Z-stack, using their tracking ID number.  

### Imports


In [ ]:
import numpy as np
import pandas as pd
import os
import time

## Set the input paths, the filename and save_file_name

The **input_path** is the top level directory where all processed subdirectories are

The **filename** is the filename in each directory that contains the data from that image set 

The **save_file_name** is what the output file will be named


In [ ]:
##CHANGE THESE
#Path to directory
input_path = '/path/to/default/Output_Folder/' #the directory where all the output subdirectories are
filename='Stardist_Labels_Enhanced_ImagesObjects_edge_size_filtered.csv'  #the file we will be looking for in each subdirectory
save_file_name  = 'Date_Meaningful_Filename_All_Channels_Output.xlsx'  #this is the output file name


In [ ]:
## Get file list
# Concatenate input_path and save_file_name
all_data_output_file = os.path.join(input_path, save_file_name)
save_path = input_path

#get a list of all the files and subdirectories in the main folder
all_files = os.listdir(input_path)

# get a list of all the subdirectories only
sub_folders = [f for f in all_files if os.path.isdir(os.path.join(input_path, f))]

In [ ]:
print("There are: ", len(sub_folders), "folders")

In [ ]:
n=1
for folder in sub_folders:
    print("run number", n)
    
    id = folder
    id_short = id[:31] if len(id) > 31 else id
    print("id_short: ", id_short)
    
    csv_input_file = os.path.join(input_path, id, filename)
    excel_out_file = f"{id}.xlsx"
    SaveFileName = os.path.join(save_path, excel_out_file)
    print("input file: ", csv_input_file)
    print("save_file_path: ", SaveFileName)
    
    ##read in the file, sometimes if coming from the server, this fails the first time, so we retry after a second
        
    try:
        filename_table = pd.read_csv(csv_input_file)
        
    except Exception:
        time.sleep(1)
        try:
            filename_table = pd.read_csv(csv_input_file)
        except Exception: #check to make sure the folder had the right file
            print("Folder: ", csv_input_file, " didn't work")
            continue
    if filename_table.empty:
        time.sleep(1)
        print("Stuck on File")
        print(csv_input_file)
        filename_table = pd.read_csv(csv_input_file)
        
    if not filename_table.empty:
    # Select specific columns for Small_Table
        cols = [
            'ImageNumber', 
            'ObjectNumber',
            'Intensity_IntegratedIntensity_C1',
            'Intensity_IntegratedIntensity_C1_Corr',
            'Intensity_IntegratedIntensity_C1_Enhanced',
            'Intensity_IntegratedIntensity_C1_Enhanced_Corr',
            'Intensity_IntegratedIntensity_C2',
            'Intensity_IntegratedIntensity_C2_Corr',
            'Intensity_IntegratedIntensity_C2_Enhanced_Corr',
            'Intensity_IntegratedIntensity_C2_enhanced',
            'Intensity_IntegratedIntensity_C3',
            'Intensity_IntegratedIntensity_C3_Corr',
            'Intensity_IntegratedIntensity_C3_Enhanced',
            'Intensity_IntegratedIntensity_C3_Enhanced_Corr',
            'Intensity_IntegratedIntensity_C4',
            'Intensity_IntegratedIntensity_C4_Corr',
            'Intensity_IntegratedIntensity_C4_Enhanced',
            'Intensity_IntegratedIntensity_C4_Enhanced_Corr',
            'Intensity_IntegratedIntensity_C5',
            'AreaShape_Eccentricity', 
            'AreaShape_Area',
            'TrackObjects_Label_2',
            'TrackObjects_Lifetime_2'
        ]
    small_table = filename_table[cols]
    tracked_objects = small_table['TrackObjects_Label_2'].dropna().unique()

    total_intensity = []
    ## the tallying section
    
    for obj_id in tracked_objects:
        obj_data = small_table[small_table['TrackObjects_Label_2'] == obj_id]

        total_intensity.append([
            obj_id,
            obj_data['Intensity_IntegratedIntensity_C1'].sum(),
            obj_data['Intensity_IntegratedIntensity_C1_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C1_Enhanced'].sum(),
            obj_data['Intensity_IntegratedIntensity_C1_Enhanced_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C2'].sum(),
            obj_data['Intensity_IntegratedIntensity_C2_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C2_Enhanced_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C2_enhanced'].sum(),
            obj_data['Intensity_IntegratedIntensity_C3'].sum(),
            obj_data['Intensity_IntegratedIntensity_C3_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C3_Enhanced'].sum(),
            obj_data['Intensity_IntegratedIntensity_C3_Enhanced_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C4'].sum(),
            obj_data['Intensity_IntegratedIntensity_C4_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C4_Enhanced'].sum(),
            obj_data['Intensity_IntegratedIntensity_C4_Enhanced_Corr'].sum(),
            obj_data['Intensity_IntegratedIntensity_C5'].sum(),
            obj_data['AreaShape_Eccentricity'].max(),
            obj_data['AreaShape_Area'].sum(),
            obj_data['TrackObjects_Lifetime_2'].max()
        ])

    column_headers = [
        'Tracked_Object_Number',
        'Intensity_IntegratedIntensity_C1',
        'Intensity_IntegratedIntensity_C1_Corr',
        'Intensity_IntegratedIntensity_C1_Enhanced',
        'Intensity_IntegratedIntensity_C1_Enhanced_Corr',
        'Intensity_IntegratedIntensity_C2',
        'Intensity_IntegratedIntensity_C2_Corr',
        'Intensity_IntegratedIntensity_C2_Enhanced_Corr',
        'Intensity_IntegratedIntensity_C2_enhanced',
        'Intensity_IntegratedIntensity_C3',
        'Intensity_IntegratedIntensity_C3_Corr',
        'Intensity_IntegratedIntensity_C3_Enhanced',
        'Intensity_IntegratedIntensity_C3_Enhanced_Corr',
        'Intensity_IntegratedIntensity_C4',
        'Intensity_IntegratedIntensity_C4_Corr',
        'Intensity_IntegratedIntensity_C4_Enhanced',
        'Intensity_IntegratedIntensity_C4_Enhanced_Corr',
        'Intensity_IntegratedIntensity_C5',
        'Max_Eccentricity',
        'Total_Area',
        'Lifetime'
    ]

    output_df = pd.DataFrame(total_intensity, columns=column_headers)
    time.sleep(2)
    output_df.to_excel(SaveFileName, index=False)
    print(SaveFileName)
    time.sleep(5)
    print("n is: ", n)
    
    
    
    
    
    if n == 1:  ##on the first iteration, create the excel file
        mode = 'w'
        with pd.ExcelWriter(all_data_output_file, mode=mode, engine='openpyxl') as writer:
            output_df.to_excel(writer, sheet_name=id_short, index=False)
   
    else:  ##append a new sheet to the excel file
        mode = 'a'
        with pd.ExcelWriter(all_data_output_file, mode=mode, engine='openpyxl', if_sheet_exists='replace') as writer:
            output_df.to_excel(writer, sheet_name=id_short, index=False)
    n=n+1 #iterate for the ID number.  This is done at the end, so that files that fail, and get bumped by the exception block don't iterate the excel file sheets, 
          # or in worse case scenario, if the first folder read fails, then this will allow us to still write the excel out